# Project: Marvel Cinematic Universe's Acquintance network


This project is about the MCU and its acquintance network created by different actors / actresses. This means that we have to find all connections, weight the network accordingly and give different measured parameters of it.

## Data Cleaning

In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl

import sys
import os
import glob
import pathlib
import codecs

import re
import uuid

Let's see how each row looks!

In [3]:
mcu_orig = pd.read_csv("data/characters.csv")
display(mcu_orig.head())
display(mcu_orig.info())

mcu_sorted0 = mcu_orig.sort_values(by = 'character')

,character,Ant-Man,Avengers: Age of Ultron,Avengers: Endgame,Avengers: Infinity War,Black Panther,Captain America: Civil War,Captain America: The First Avenger,Captain America: The Winter Soldier,Captain Marvel,...,Spider-Man: Homecoming,The Avengers,Thor,Thor: Ragnarok,Thor: The Dark World,total lines,total words,movie appearances,avg words per line,avg lines per movie
0,TONY STARK,0,173,137,119,0,165,0,0,0,...,74,152,0,0,0,1788,22590,9,12,198
1,STEVE ROGERS,3,133,119,25,0,154,186,218,3,...,7,121,0,0,0,977,8653,10,8,97
2,THOR,0,55,70,74,0,0,0,0,0,...,0,47,166,289,133,863,9487,7,10,123
3,PETER PARKER,0,0,7,39,0,73,0,0,0,...,402,0,0,0,0,551,5517,4,10,137
4,NATASHA ROMANOFF,1,79,69,15,0,64,0,112,3,...,0,97,0,1,0,498,4689,10,9,49


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 652 entries, 0 to 651
Data columns (total 24 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   character                            652 non-null    object
 1   Ant-Man                              652 non-null    int64 
 2   Avengers: Age of Ultron              652 non-null    int64 
 3   Avengers: Endgame                    652 non-null    int64 
 4   Avengers: Infinity War               652 non-null    int64 
 5   Black Panther                        652 non-null    int64 
 6   Captain America: Civil War           652 non-null    int64 
 7   Captain America: The First Avenger   652 non-null    int64 
 8   Captain America: The Winter Soldier  652 non-null    int64 
 9   Captain Marvel                       652 non-null    int64 
 10  Guardians of the Galaxy Vol. 2       652 non-null    int64 
 11  Iron Man                             652 non-

None

In [4]:
display(mcu_sorted0['character'].values)

array(['25TH FLOOR STRIKE AGENT', '4F DOCTOR', '911 OPERATOR',
       'A1 CLINT BARTON', 'A1 HULK', 'A1 NATASHA', 'A1 STEVE', 'A1 THOR',
       'A1 TONY', 'AARON DAVIS', 'ABRAHAM BROWN', 'ABU', 'ACCUSER',
       'ACTOR LOKI', 'ACTOR ODIN', 'ACTOR SIF', 'ACTOR THOR', 'ADMIRAL',
       'ADMISSIONS NURSE', 'ADRIAN TOOMES', 'AGENT', 'AGENT CALE',
       'AGENT DELANCEY', 'AGENT GARRETT', 'AGENT JACKSON', 'AKIHIKO',
       'ALDRICH KILLIAN', 'ALETA', 'ALEXANDER PIERCE', 'ALGRIM',
       'ALPHA GUARD', 'ANCIENT ONE', 'ANNE MARIE HOAG', 'ANNIE',
       'ANNOUNCEMENT', 'ANNOUNCER', 'ANTIQUE STORE OWNER', 'ANTON VANKO',
       'APPLE EMPLOYEE', 'ARMY HECKLER#1', 'ARMY HECKLER#2',
       'ARMY HECKLER#3', 'ARMY HECKLER#4', 'ASGARDIAN CITIZENS',
       'ASGARDIAN EINHERJAR', 'ASGARDIAN MAN', 'ASGARDIAN NURSE',
       'ASGARDIAN PA', 'ASGARDIAN WOMAN', 'ATT-LASS', 'ATTACHE',
       'ATTENDANT', 'AVENGERS JIMMY', 'AVENGERS KID', 'AYESHA', 'AYO',
       'BAD NEBULA', 'BAILIFF', 'BALLET INSTRUCTOR', 

**Problem:**

It seems like multiple rows are in here that has multiple characters in it. This may mean that these chaacters are actually talking together. These have to be separated into rows and it should be compared if there are multiple rows that actually match. **THIS MEANS I HAVE TO READ THROUGH IT MANUALLY**

In [5]:
mcu_sorted0 = mcu_orig.sort_values(by = 'character')

In [6]:
char_list = [",", "/"]  #chars to match
multiple_ppl = []
for i in mcu_sorted0['character'].values:
    multiple_ppl.append(any([characters in char_list for characters in i]))

print(len(mcu_sorted0['character'].values[multiple_ppl]))
print(mcu_sorted0['character'].values[multiple_ppl])

10
['ELDERS, WARRIORS, DORA' 'JASON/BETTY' 'JOSEF / SUPER SOLDIER #1'
 'LIZ/NED' 'LOKI/STEVE' 'SHURI/NAKIA' 'THOR / BANNER' 'THOR/SELVIG'
 'VERS, BRON-CHAR, KORATH, MINN-ERVA AND ATT-LASS' 'VOLSTAGG / HOGUN']


This means there are 10 rows depicting multiple people as on character, because that's how the transcript has it. It must be expected that these people may not be here individually.

**It doesn't matter how much I want to avoid doing this manually, I have to do it manually** (technically a robust regex could handle it)

In [7]:
#this means that i have to add them manually
any(mcu_sorted0['character'].values == 'ELDERS')

False

In [8]:
my_list = [['ELDERS','WARRIORS','DORA'],
           ['JASON','BETTY'],
           ['JOSEF', 'SUPER SOLDIER #1'],
           ['LIZ', 'NED'],
           ['LOKI','STEVE'],
           ['SHURI', 'NAKIA'],
           ['THOR', 'BANNER'],
           ['THOR','SELVIG'],
           ['VERS', 'BRON-CHAR', 'KORATH', 'MINN-ERVA', 'ATT-LASS'],
           ['VOLSTAGG', 'HOGUN']]

In [9]:
mcu_sorted1 = mcu_sorted0.copy()
"""
for i in range(len(my_list)):
    #get loc for values
    loc = np.argmax(mcu_sorted0['character'].values == mcu_sorted0['character'][multiple_ppl].values[i])
    values0 = mcu_sorted0.iloc[loc].values
    print("\n",loc,values)
    for j in my_list[i]:
        #check if its in
        _isin = any(mcu_sorted0['character'].values == j)

        if not _isin:
            #if not in add
            temp =np.append(np.array(j),(values0[1:]))
            mcu_sorted1.loc[len(mcu_sorted1.index)] = temp
            print('Adding ',temp)
            
        else:
            #if in, modify by getting old values and adding them.
            loc2 = np.argmax(mcu_sorted0['character'].values == j)
            val2 = np.array(mcu_sorted0.iloc[loc2].values[1:])+ np.array(values0[1:])
            inserting =  np.append(np.array(j),(val2))
            print('Modifying ',inserting)
            mcu_sorted1.iloc[loc2] = inserting         
"""            
#go through the values and drop the rows that has multiple people in it
new_mul_ppl = []
for i in mcu_sorted1['character'].values:
    new_mul_ppl.append(any([characters in char_list for characters in i]))

mcu_sorted1 = mcu_sorted1[~np.array(new_mul_ppl)]

#drop columns tha are not needed

mcu_sorted1 = mcu_sorted1.drop(columns = mcu_sorted0.columns[-5:])

In [10]:
mcu_sorted1

,character,Ant-Man,Avengers: Age of Ultron,Avengers: Endgame,Avengers: Infinity War,Black Panther,Captain America: Civil War,Captain America: The First Avenger,Captain America: The Winter Soldier,Captain Marvel,Guardians of the Galaxy Vol. 2,Iron Man,Iron Man 2,Iron Man 3,Spider-Man: Homecoming,The Avengers,Thor,Thor: Ragnarok,Thor: The Dark World
500,25TH FLOOR STRIKE AGENT,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
192,4F DOCTOR,0,0,0,0,0,0,7,0,0,0,0,0,0,0,0,0,0,0
556,911 OPERATOR,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
477,A1 CLINT BARTON,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
417,A1 HULK,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,YOUNG TONY STARK,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0
230,YOUNG ZURI,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0
338,ZEMO'S WIFE,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0
531,ZRINKA,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


I am idiot, as I forgot that hte people that talk together normally have their own individual lines, thus i don't need to play with these... just remove them.

But there are with multiple names, so these have to be combined

In [11]:
display(mcu_sorted1.iloc[np.argmax(mcu_sorted1['character'] == 'A1 THOR')])
display(mcu_sorted1.iloc[np.argmax(mcu_sorted1['character'] == 'THOR')])
display(mcu_sorted1.iloc[np.argmax(mcu_sorted1['character'] == 'ACTOR THOR')])

character                              A1 THOR
Ant-Man                                      0
Avengers: Age of Ultron                      0
Avengers: Endgame                            5
Avengers: Infinity War                       0
Black Panther                                0
Captain America: Civil War                   0
Captain America: The First Avenger           0
Captain America: The Winter Soldier          0
Captain Marvel                               0
Guardians of the Galaxy Vol. 2               0
Iron Man                                     0
Iron Man 2                                   0
Iron Man 3                                   0
Spider-Man: Homecoming                       0
The Avengers                                 0
Thor                                         0
Thor: Ragnarok                               0
Thor: The Dark World                         0
Name: 221, dtype: object

character                              THOR
Ant-Man                                   0
Avengers: Age of Ultron                  55
Avengers: Endgame                        70
Avengers: Infinity War                   74
Black Panther                             0
Captain America: Civil War                0
Captain America: The First Avenger        0
Captain America: The Winter Soldier       0
Captain Marvel                            0
Guardians of the Galaxy Vol. 2            0
Iron Man                                  0
Iron Man 2                                0
Iron Man 3                                0
Spider-Man: Homecoming                    0
The Avengers                             47
Thor                                    166
Thor: Ragnarok                          289
Thor: The Dark World                    133
Name: 2, dtype: object

character                              ACTOR THOR
Ant-Man                                         0
Avengers: Age of Ultron                         0
Avengers: Endgame                               0
Avengers: Infinity War                          0
Black Panther                                   0
Captain America: Civil War                      0
Captain America: The First Avenger              0
Captain America: The Winter Soldier             0
Captain Marvel                                  0
Guardians of the Galaxy Vol. 2                  0
Iron Man                                        0
Iron Man 2                                      0
Iron Man 3                                      0
Spider-Man: Homecoming                          0
The Avengers                                    0
Thor                                            0
Thor: Ragnarok                                 12
Thor: The Dark World                            0
Name: 143, dtype: object

In [12]:
collect_dropping = ['A1 CLINT BARTON',
                    'A1 HULK',
                    'A1 NATASHA',
                    'A1 STEVE',
                    'A1 THOR',
                    'A1 TONY',             #A1 is some alternative version of the avengers
                    'BOTH',                 #no clue who is this
                   ]
for i in collect_dropping:
    mcu_sorted1 = mcu_sorted1[~(mcu_sorted1['character'] == i)]
    
display(mcu_sorted1['character'].values)

array(['25TH FLOOR STRIKE AGENT', '4F DOCTOR', '911 OPERATOR',
       'AARON DAVIS', 'ABRAHAM BROWN', 'ABU', 'ACCUSER', 'ACTOR LOKI',
       'ACTOR ODIN', 'ACTOR SIF', 'ACTOR THOR', 'ADMIRAL',
       'ADMISSIONS NURSE', 'ADRIAN TOOMES', 'AGENT', 'AGENT CALE',
       'AGENT DELANCEY', 'AGENT GARRETT', 'AGENT JACKSON', 'AKIHIKO',
       'ALDRICH KILLIAN', 'ALETA', 'ALEXANDER PIERCE', 'ALGRIM',
       'ALPHA GUARD', 'ANCIENT ONE', 'ANNE MARIE HOAG', 'ANNIE',
       'ANNOUNCEMENT', 'ANNOUNCER', 'ANTIQUE STORE OWNER', 'ANTON VANKO',
       'APPLE EMPLOYEE', 'ARMY HECKLER#1', 'ARMY HECKLER#2',
       'ARMY HECKLER#3', 'ARMY HECKLER#4', 'ASGARDIAN CITIZENS',
       'ASGARDIAN EINHERJAR', 'ASGARDIAN MAN', 'ASGARDIAN NURSE',
       'ASGARDIAN PA', 'ASGARDIAN WOMAN', 'ATT-LASS', 'ATTACHE',
       'ATTENDANT', 'AVENGERS JIMMY', 'AVENGERS KID', 'AYESHA', 'AYO',
       'BAD NEBULA', 'BAILIFF', 'BALLET INSTRUCTOR', 'BARKEEPER',
       'BARON WOLFGANG VON STRUCKER', 'BARTENDER', 'BBC ANCHOR',
       

## Creating the linklist 

In [13]:
for i in mcu_sorted1.columns[1:]:
    print(i)

Ant-Man
Avengers: Age of Ultron
Avengers: Endgame
Avengers: Infinity War
Black Panther
Captain America: Civil War
Captain America: The First Avenger
Captain America: The Winter Soldier
Captain Marvel
Guardians of the Galaxy Vol. 2
Iron Man
Iron Man 2
Iron Man 3
Spider-Man: Homecoming
The Avengers
Thor
Thor: Ragnarok
Thor: The Dark World


In [14]:
mcu_sorted1['character'][mcu_sorted1['Ant-Man'] > 0]

327                 ALPHA GUARD
583                  CAB DRIVER
90                  CASSIE LANG
205                    COMPUTER
490              COP ON SPEAKER
149                        DALE
63                 DARREN CROSS
83                         DAVE
308                   DETECTIVE
247                       FRANK
273                        GALE
20                     HANK PYM
406              HIDEOUS RABBIT
34                HOPE VAN DYNE
54                 HOWARD STARK
299    ICE CREAM STORE CUSTOMER
68           JAMES "JIM" PAXTON
89                         KURT
36                         LUIS
129                 MAGGIE LANG
160             MITCHELL CARSON
4              NATASHA ROMANOFF
317                      PEACHY
32                 PEGGY CARTER
483                POLICE RADIO
622                POOL BBQ DAD
575           PYM TECH EMPLOYEE
623         PYM TECH GATE GUARD
409     PYM TECH SECURITY GUARD
16                   SAM WILSON
9                    SCOTT LANG
1       

This seems to be correct, now I just need to create the linklist, with **UUID**. The UUID is crated by the name as a SHA-1 hash, so they are not obvious how these are choosen and i don't like random.

In [15]:
uuid.uuid5(uuid.NAMESPACE_DNS, 'VIDEO VOICE OVER')

uuids = []

for i in mcu_sorted1['character']:
    uuids.append(str(uuid.uuid5(uuid.NAMESPACE_DNS, i)))
    
display(uuids[:5])

mcu_sorted1['UUID'] = uuids

#add

['a15f6df4-3726-5e57-b049-3e0d0de031d4',
 '9e255c6b-966a-524b-a69b-81d81550f623',
 '3ad52c61-23c9-5aa8-bc05-cb5573504d9f',
 'c3870cc6-04d3-5cf4-8454-29b8dc0ab64f',
 '2da415ec-683d-5134-aba9-d059701f5e40']

In [16]:
#save to have it as a decoding table
mcu_decode = mcu_sorted1[['character', 'UUID']]

In [17]:
print(mcu_sorted1.columns[1:-1])

Index(['Ant-Man', 'Avengers: Age of Ultron', 'Avengers: Endgame',
       'Avengers: Infinity War', 'Black Panther', 'Captain America: Civil War',
       'Captain America: The First Avenger',
       'Captain America: The Winter Soldier', 'Captain Marvel',
       'Guardians of the Galaxy Vol. 2', 'Iron Man', 'Iron Man 2',
       'Iron Man 3', 'Spider-Man: Homecoming', 'The Avengers', 'Thor',
       'Thor: Ragnarok', 'Thor: The Dark World'],
      dtype='object')


In [18]:
test_ll = []

temp_vals = mcu_sorted1['UUID'][mcu_sorted1['Ant-Man'] > 0].values
print(temp_vals)
for i in range(len(temp_vals)):
    for j in range(len(temp_vals[(i+1):])):
        test_ll.append([temp_vals[i], temp_vals[j]])
            
print(len(temp_vals)); print(len(test_ll));  print(len(temp_vals)*(len(temp_vals)-1)/2)

['a925ecc2-c223-5555-b66e-0b5b24122201'
 'd940eec4-2ed8-58e9-b783-f6afd36dfa93'
 'ee9c03ac-4251-5042-ac85-986db9228a17'
 'd31a4b04-d2b4-54b6-8a61-ff1ab9173cbe'
 'ea69d1bb-ebe2-5ecf-948e-1f1e887ffef0'
 'ec544454-ac52-5f6d-b435-8797269e6c2f'
 '7e3723ff-e17a-5c64-8958-6980ce5212bf'
 '5fdf183e-ffd5-5818-981d-5e71d1875762'
 'cdb90dad-2d2e-50f8-bfba-10cc5ee33aec'
 '0a4421f6-cb4b-5347-a035-a4b389dde17b'
 '3432c3c3-735b-5827-8b25-152f326bb95a'
 '550d75ab-33f0-5cdb-be9b-1fbeb918a224'
 '90d48a51-6178-58bf-b154-b84a9132809a'
 '52dbb287-a102-540a-a339-cd2a6c631ef4'
 '5fd74e70-e177-5f74-914c-7ee6756e9615'
 'cecdfd88-cfa5-5be8-8c86-95656571e58c'
 '1f4161c2-5d2b-522e-a1cb-af76d637423b'
 '9ace14f8-ab60-5ed0-9dab-f6cf7e1e492d'
 'a61f7564-b8f8-5601-9ac0-50bb648852b3'
 '2ec35106-b382-5af3-980d-54a4035e9b50'
 '62d468d4-86c8-530d-b71f-e34a3a6ba5bf'
 'a63edbe6-0772-5369-addf-260df61caa93'
 '19fd9a03-ee7d-5b19-88f1-47a4bce11af7'
 'd5f7a64b-6e8e-5b1a-8248-0c1d3c25c080'
 '54092dbd-f41f-570e-94f0-39cee581eaf8'


In [19]:
linklist = []

for k in mcu_sorted1.columns[1:-1]:
    temp_vals = mcu_sorted1['UUID'][mcu_sorted1[k] > 0].values
    for i in range(len(temp_vals)):
        for j in range(len(temp_vals[(i+1):])):
            #ALPHA GUARD IS IN THE SAME FILM AS ALPHA GUARD
            #adjecancy matrix diagonal elements
            linklist.append([temp_vals[i], temp_vals[j]])
        
display(len(linklist))

22601

Okey, it seems like I have a network with **22601** connections. Some of them is weighted over 1. E.g.: Thor and Hulk is in more movies together than one. The full graph would be **3 395 104** connection. (fully connected graph for each movie, every connection (including self loops) is weighted to 17). Let's save it and the decoding table!

**AND WE ARE FINISHED**

No we are not. Actually the problem was shown by `NETWORKX`. As I was totally able to forgot, some people are in multiple films together, causing the network to be weightned if someone doesnt watch out for this. But I need an unweighted network, so I need to sort this out. For my luck, if I give networkx a linklist, it creates the unweigthed network with ease. Using its edges, I can see how many links with weight one exists.

In [21]:
uuids, counts = np.unique(np.array(linklist), return_counts=True)
print(max(counts))

602


In [22]:
import networkx as nx

In [25]:
G = nx.Graph()
G.add_edges_from(linklist)

In [26]:
G.edges()

EdgeView([('a925ecc2-c223-5555-b66e-0b5b24122201', 'a925ecc2-c223-5555-b66e-0b5b24122201'), ('a925ecc2-c223-5555-b66e-0b5b24122201', 'd940eec4-2ed8-58e9-b783-f6afd36dfa93'), ('a925ecc2-c223-5555-b66e-0b5b24122201', 'ee9c03ac-4251-5042-ac85-986db9228a17'), ('a925ecc2-c223-5555-b66e-0b5b24122201', 'd31a4b04-d2b4-54b6-8a61-ff1ab9173cbe'), ('a925ecc2-c223-5555-b66e-0b5b24122201', 'ea69d1bb-ebe2-5ecf-948e-1f1e887ffef0'), ('a925ecc2-c223-5555-b66e-0b5b24122201', 'ec544454-ac52-5f6d-b435-8797269e6c2f'), ('a925ecc2-c223-5555-b66e-0b5b24122201', '7e3723ff-e17a-5c64-8958-6980ce5212bf'), ('a925ecc2-c223-5555-b66e-0b5b24122201', '5fdf183e-ffd5-5818-981d-5e71d1875762'), ('a925ecc2-c223-5555-b66e-0b5b24122201', 'cdb90dad-2d2e-50f8-bfba-10cc5ee33aec'), ('a925ecc2-c223-5555-b66e-0b5b24122201', '0a4421f6-cb4b-5347-a035-a4b389dde17b'), ('a925ecc2-c223-5555-b66e-0b5b24122201', '3432c3c3-735b-5827-8b25-152f326bb95a'), ('a925ecc2-c223-5555-b66e-0b5b24122201', '550d75ab-33f0-5cdb-be9b-1fbeb918a224'), ('a925

In [34]:
final_linklist = []

count = 0
for i in range(len(G.edges())):
    final_linklist.append(list(list(G.edges())[i]))
    count = count + 1
count

10966

In [41]:
file1 = codecs.open("data/_decoding.csv","w", "utf-8")
file2 = codecs.open("data/_linklist.csv","w", "utf-8")

file1.write("character,uuid\n")
file2.write("uuid1,uuid2\n")

file1.close()
file2.close()

file1 = open("data/_decoding.csv", 'a')
for i in mcu_decode.values:
    file1.write(i[0])
    file1.write(',')
    file1.write(i[1])
    file1.write('\n')
    
file1.close()

count = 0
file2 = open("data/_linklist.csv", 'a')
for i in final_linklist:
    if i[0] != i[1]:
        count= count + 1
        file2.write(i[0])
        file2.write(',')
        file2.write(i[1])
        file2.write('\n')
    
file2.close()

print("Rows in linklist:" + str(count))

Rows in linklist:10628


**And it is finalised and done!**

In [40]:
!git push origin master

To https://github.com/AdamGTaylor/Datamodelling_2021
   539ebe8..b803017  master -> master
